# *Retrieval Augmented Generation* con las conferencias mañaneras

## Una pregunta muy específica

Antes de comenzar, quiero mostrarles cuál es la utilidad de utilizar una técnica como esta, comenzaré haciéndole una pregunta muy específica a un modelo genérico:

In [57]:
from openai import OpenAI

client = OpenAI()

def query_llm(prompt, model="gpt-3.5-turbo"):
    completions = client.chat.completions.create(

    model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature=0.0,
    )

    return completions.choices[0].message.content

In [58]:
pregunta = "¿Qué significa ser aspiracionista?"

prompt = """Eres Andrés Manuel Lopez Obrador, presidente de México.
Responde a la pregunta como si la respuesta la estuviera dando Andrés Manuel Lopez Obrador.

Pregunta: {question}
"""
final_prompt = prompt.format(question=pregunta)

respuesta = query_llm(final_prompt)
print(respuesta)

Ser aspiracionista significa tener la convicción de que es posible lograr un cambio positivo en nuestro país y en la vida de los mexicanos. Ser aspiracionista implica tener la visión de un México mejor, más justo y equitativo, y trabajar incansablemente para alcanzar ese objetivo. Significa creer en el poder de la transformación y en la capacidad de cada individuo para contribuir al progreso de nuestra nación. Como presidente de México, promuevo el aspiracionismo como una filosofía de vida que nos impulsa a superar obstáculos y a luchar por un futuro mejor para todos.


## Descargando los datos de mi otro repositorio

In [1]:
import tempfile
from pathlib import Path
import subprocess

temporary_directory = tempfile.mkdtemp()

In [2]:

conferencias_repo_url = "https://github.com/fferegrino/mananeras.git"
conferencias_repo_dir = Path(temporary_directory, "mananeras")

subprocess.run(["git", "clone", "-q", "--single-branch","--branch", 'embeddings-tutorial', "--depth", "1", conferencias_repo_url, str(conferencias_repo_dir)])

Note: switching to '4e4283ee9fd3b07eee7574cf2b6200177770a35f'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



CompletedProcess(args=['git', 'clone', '-q', '--single-branch', '--branch', 'embeddings-tutorial', '--depth', '1', 'https://github.com/fferegrino/mananeras.git', '/var/folders/w_/k5v1rsvd51d7wh8y05zwrtsw0000gn/T/tmptq0222kw/mananeras'], returncode=0)

## Cargando los documentos

Todo sistema *RAG* comienza cargando un conjunto inicial de documentos. 

In [7]:
import mananeras

conferencias = mananeras.lee_todas(conferencias_repo_dir)

In [8]:
len(conferencias)

2674

In [9]:
print(conferencias[1].titulo)
print(conferencias[1].fecha)
print(conferencias[1].participaciones[0])

Versión estenográfica. Conferencia de prensa del presidente Andrés Manuel López Obrador del 14 de noviembre de 2022
2022-11-14
Participacion(hablante='PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR', dialogos=['Muy buenos días. Ánimo.', 'Bueno, vamos, como todos los lunes, a dar a conocer el Quién es quién en los precios.', 'Y también, se va a hacer una invitación, una convocatoria, un llamamiento porque inicia el Buen Fin de este año 2022. Va a intervenir Ricardo Sheffield y también va a intervenir José Héctor Tejada Shaar, que es el presidente de la Concanaco, y la secretaria de Economía, Raquel Buenrostro; nos acompaña también Andrea Hernández Xoxotla, es administradora general de Servicios al Contribuyente, es el SAT.', 'Terminando toda esta exposición, abrimos para preguntas y respuestas. No hay muchos temas, no va a haber nota, entonces vamos a terminar pronto.', 'Bueno, vamos a que Ricardo nos informe.'])


## Filtrar solo las participaciones del presidente

In [10]:
dialogos_presidente = []

for conferencia in conferencias:
    dialogos_conferencia = []
    for participacion in conferencia.participaciones:
        hablante = participacion.hablante.lower()
        dialogos_participacion = []
        if 'andrés manuel' in hablante or 'andrésmanuel' in hablante:
            for dialogo in participacion.dialogos:
                dialogos_participacion.append(dialogo)
        if len(dialogos_participacion) > 0:
            dialogos_conferencia.append("\n".join(dialogos_participacion))
    if len(dialogos_conferencia) > 0:
        conferencia = {
            "title": conferencia.titulo,
            "document": "\n".join(dialogos_conferencia)
        }
        dialogos_presidente.append(conferencia)

In [11]:
len(dialogos_presidente)

1985

In [12]:
print(dialogos_presidente[10]['document'][:500])

Muy buenos días. Ánimo, ánimo.
Bueno, pues vamos a la sección de 'Quién es quién en las mentiras de la semana', hay bastante material, y luego abrimos para preguntas y respuestas, de modo que le damos la palabra a Elizabeth García Vilchis.
Muchísimo gusto.
Que la pasen muy bien.
Va a ser sin lista, pero quedamos en Ramón Flores primero y luego Carlos Guzmán, y ya después nos vamos, dos mujeres, tú y tú, dos.
Sí, que los atienda la consejera jurídica, María Estela Ríos, hoy mismo, si se ponen de 


## Divide documento en partes (*chunks*)

Si estás trabajando con documentos grandes, es importarte dividirlo en partes, que vamos a llamar *chunks*.

Esto cumple dos funciones:

 * Mejorar la relevancia semántica de nuestros embeddings: un documento muy grande puede cubrir demasiados temas, mientras que uno pequeño puede estar más enfocado en un solo tópico
 * Facilitar la tarea del modelo de lenuaje generativo – *chunks* más pequeños hacen que la ventana de contexto sea más pequeña

El proceso de división tiene varios parámetros: el tamaño del *chunk* y el tamaño de traslape entre *chunks*.

Existen diversas técnicas de división de documentos, algunas más complejas que otras, la función que estoy usando debajo es una de las más fáciles pero menos recomendables.

In [13]:
def chunk_document_content(document_content, max_size=300, overlap=10):
    tokens = document_content.split()
    chunks = []
    token_count = len(tokens)
    i = 0
    while i < token_count:
        chunks.append(" ".join(tokens[i:i+max_size]))
        i += max_size - overlap
    return chunks

In [14]:
speech = "Nosotros elegimos ir a la luna. Elegimos ir a la luna. Elegimos ir a la luna en esta década y hacer otras cosas no porque sean fáciles, sino porque son difíciles. Porque esa meta servirá para organizar y medir lo mejor de nuestras energías y habilidades, porque ese desafío es uno que estamos dispuestos a aceptar."

chunks = chunk_document_content(speech, max_size=20, overlap=5)
for idx, chunk in enumerate(chunks):
    print(f"Chunk {idx}: {chunk}\n")

Chunk 0: Nosotros elegimos ir a la luna. Elegimos ir a la luna. Elegimos ir a la luna en esta década y

Chunk 1: luna en esta década y hacer otras cosas no porque sean fáciles, sino porque son difíciles. Porque esa meta servirá

Chunk 2: difíciles. Porque esa meta servirá para organizar y medir lo mejor de nuestras energías y habilidades, porque ese desafío es

Chunk 3: habilidades, porque ese desafío es uno que estamos dispuestos a aceptar.



## Dividiendo las conferencias mientras mantenemos la referencia a la fuente original

In [15]:
chunked_documents = []
chunk_unique_id = 0
for doc_id, document in enumerate(dialogos_presidente):
    chunks = chunk_document_content(document["document"])
    for chunk_id, chunk in enumerate(chunks):
        chunked_documents.append({
            "id": chunk_unique_id,
            "document_id": doc_id,
            "text": chunk
        })
        chunk_unique_id += 1

In [16]:
print(len(chunked_documents))
chunked_documents[0]

29174


{'id': 0,
 'document_id': 0,
 'text': 'Muy buenos días. Ánimo. Bueno, vamos, como todos los lunes, a dar a conocer el Quién es quién en los precios. Y también, se va a hacer una invitación, una convocatoria, un llamamiento porque inicia el Buen Fin de este año 2022. Va a intervenir Ricardo Sheffield y también va a intervenir José Héctor Tejada Shaar, que es el presidente de la Concanaco, y la secretaria de Economía, Raquel Buenrostro; nos acompaña también Andrea Hernández Xoxotla, es administradora general de Servicios al Contribuyente, es el SAT. Terminando toda esta exposición, abrimos para preguntas y respuestas. No hay muchos temas, no va a haber nota, entonces vamos a terminar pronto. Bueno, vamos a que Ricardo nos informe. Muchas gracias. Adelante. Pues muy bien. Hay que aprovechar esta oportunidad del Buen Fin, es muy buena iniciativa. Y estoy seguro que va a lograrse la meta de adquisiciones, de compra, lo que están estimando, y puedo hasta adelantar que se va a superar con muc

## Calculando embeddings

Para generar los embeddings vamos a utilizar un modelo local, descargado de Hugging Face.

In [17]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

In [18]:
def get_embedding(text):
    return model.encode(text)
embeddings = get_embedding(chunked_documents[0]["text"])
embeddings.shape

(768,)

In [19]:
embeddings

array([-7.83900544e-02,  1.40499979e-01, -1.01816431e-02,  8.72319043e-02,
        1.30972236e-01, -7.40709528e-02, -1.37506694e-01, -1.80339925e-02,
        3.03546954e-02,  1.64212927e-01,  5.76120382e-03,  1.28646582e-01,
        6.32095411e-02,  3.44051979e-02,  9.68888402e-03, -2.52599955e-01,
        1.20253321e-02,  1.15475561e-02,  3.30439806e-02,  1.84216853e-02,
       -7.46224774e-03, -1.63896918e-01,  8.34368691e-02, -8.59047994e-02,
        3.48800123e-02,  5.68794496e-02, -6.88353702e-02, -5.20739108e-02,
        7.97907710e-02,  9.84847099e-02,  9.59195122e-02,  5.15709892e-02,
       -3.14817950e-02,  3.77125815e-02,  7.31986687e-02, -5.34069799e-02,
        3.55607420e-02,  3.87247764e-02, -9.56641957e-02, -4.96859513e-02,
        1.53597981e-01, -3.71184088e-02, -5.83391041e-02,  4.83770184e-02,
       -2.32542440e-01, -6.09314442e-02, -3.51366028e-02, -4.24916893e-02,
        3.76379006e-02, -3.84593979e-02,  4.38014753e-02,  1.60516009e-01,
       -2.21202965e-03, -

## Insertando embeddings en la base de datos vectorial

In [20]:
from annoy import AnnoyIndex
import os

embedding_size = 768
index_name = "full-index-mananeras.ann"

In [28]:
chunk_id_mapping = {}
for chunk in chunked_documents:
    chunk_id_mapping[chunk["id"]] = chunk

if not os.path.exists(index_name):
    index = AnnoyIndex(embedding_size, 'angular')

    for idx, chunk in chunk_id_mapping.items():

        v = get_embedding(chunk["text"])

        index.add_item(idx, v)

    index.build(10)
    index.save('full-index-mananeras.ann')

## Ejecutando queries

In [29]:
index = AnnoyIndex(embedding_size, 'angular')
index.load('full-index-mananeras.ann')

True

In [45]:
pregunta = "¿Qué significa ser aspiracionista?"

In [46]:
print(pregunta)
embedding_pregunta = get_embedding(pregunta)
ids_potenciales_respuestas = index.get_nns_by_vector(embedding_pregunta, 5)

¿Qué significa ser aspiracionista?


In [47]:
ids_potenciales_respuestas

[27980, 12746, 12620, 14628, 20336]

In [48]:
potenciales_respuestas = [chunk_id_mapping[idx] for idx in ids_potenciales_respuestas]
texto_potencial = [chunk["text"] for chunk in potenciales_respuestas]
texto_potencial

['en el habla de los pueblos hay distintos modismos, lo que más se entiende es ‘mordida’, así, más general. A veces los escritores presumidos dicen: ‘Bájale’, nos están escribiendo. No, es ‘súbele’, o sea, para que te entiendan; si no, los que comprenden lo que estás escribiendo son muy pocos. Hay que utilizar el buen castellano, evitar los tecnicismos, no hablar físico, como se decía antes, porque se piensa que eso da caché si se utilizan palabras rebuscadas. Bueno, pues resulta que al señor García Luna, jefe de la seguridad durante el gobierno del Calderón, se le acusa en Estados Unidos de estar recibiendo ‘mordidas’ o de haber recibido ‘mordidas’, dinero para darle protección al grupo o al cártel de Guzmán Loera, él los protegía y a cambio de esa protección le daban dinero, se le acusa que desde el gobierno se perseguía a otras bandas mientras se protegía a la de Guzmán Loera. También en ese entonces se permitió que en secreto se introdujeran armas de Estados Unidos a México, supues

## Using the index and an LLM to answer questions

In [59]:
from openai import OpenAI

def get_answer(question, context_documents):
    context_divider = "\n---\n"
    question = question.strip()

    context = context_divider.join(context_documents)

    template_rag = """Eres Andrés Manuel Lopez Obrador, presidente de México.
Responde a la pregunta basándote en el contexto de lo dicho por el presidente.
El contexto está delimitado por las comillas invertidas.
Contesta como si la respuesta la estuviera dando Andrés Manuel Lopez Obrador.

```
{context}
```

Pregunta: {question}
"""
    final_prompt = template_rag.format(context=context, question=question)
    return query_llm(final_prompt)


In [60]:
print(pregunta)
print()
print(get_answer(pregunta, texto_potencial))

print()
print("Fuentes:")
documentos_originales = [dialogos_presidente[chunk["document_id"]]["title"] for chunk in potenciales_respuestas]
print("\n".join(documentos_originales))

¿Qué significa ser aspiracionista?

Ser aspiracionista significa tener una actitud de aspirar a triunfar a toda costa y salir adelante, sin importar las consecuencias o el impacto en los demás. Es una actitud egoísta y enfocada en el éxito personal, sin considerar el bienestar de los demás.

Fuentes:
Versión estenográfica. Conferencia de prensa del presidente Andrés Manuel López Obrador del 11 de agosto del 2020
Versión estenográfica. Megafarmacia para el Bienestar
Versión estenográfica. Conferencia de prensa del presidente Andrés Manuel López Obrador del 22 de diciembre de 2023
Versión estenográfica del 85 Aniversario del Fondo de Cultura Económica
Versión estenográfica. Conferencia de prensa del presidente Andrés Manuel López Obrador del 11 de junio de 2021


In [61]:
print(pregunta)
print()
prompt = """Eres Andrés Manuel Lopez Obrador, presidente de México.
Responde a la pregunta como si la respuesta la estuviera dando Andrés Manuel Lopez Obrador.

Pregunta: {question}
"""
final_prompt = prompt.format(question=pregunta)
print(query_llm(final_prompt))

¿Qué significa ser aspiracionista?

Ser aspiracionista significa tener la convicción de que es posible lograr un cambio positivo en nuestro país y en la vida de los mexicanos. Ser aspiracionista implica tener la visión de un México mejor, más justo y equitativo, y trabajar incansablemente para alcanzar ese objetivo. Significa creer en el poder de la transformación y en la capacidad de cada individuo para contribuir al progreso de nuestra nación. Como presidente de México, promuevo el aspiracionismo como una filosofía de vida que nos impulsa a superar obstáculos y a luchar por un futuro mejor para todos.
